In [1]:
import pandas as pd
import numpy as np

Load in `.csv` file that came from `stage-p2x`

In [2]:
df = pd.read_csv('runs_with_eval_loss_and_params.csv')

Load in data from MolNet finetuning

In [3]:
import s3fs
import os

In [4]:
fs = s3fs.S3FileSystem()

In [5]:
import os
import json

In [1]:
model_bucket = ""

In [6]:
cloud_dir = f's3://{model_bucket}/chemberta/mtr_pretraining_5M_20210804/molnet_mtr_5M_ft_20210804/'

In [7]:
def get_dataframes(cloud_dir):
    run_dirs = fs.ls(cloud_dir)
    data_avg = []
    df_all = pd.DataFrame()
    for rd in run_dirs:
        run_name = os.path.basename(os.path.normpath(rd))
        # go one level down to get the molnet task
        molnet_task_data_avg = {}
        molnet_task_data_all = {}
        for molnet_task_dir in fs.ls(rd):
            molnet_task_name = os.path.basename(os.path.normpath(molnet_task_dir))
            results_dir = os.path.join(molnet_task_dir, "results/")
            for subset in ["valid", "test"]:
                with fs.open(os.path.join(results_dir, subset, "metrics.json")) as f:
                    metrics = json.load(f)
                # pick first item to get the keys
                metric_names = list(list(metrics.items())[0][1].keys())
                metric_res = {mn: [] for mn in metric_names}
                for seed, res in metrics.items():
                    for mn, mres in res.items():
                        if mn == "pearsonr":
                            metric_res[mn].append(mres[0])
                        else:
                            metric_res[mn].append(mres)
                molnet_task_data_all.update({f"{molnet_task_name}_{subset}_{mn}": metric_res[mn] for mn in metric_names})
                average_metrics = {f"{molnet_task_name}_{subset}_{mn}_mean": np.mean(metric_res[mn]) for mn in metric_names}
                std_metrics = {f"{molnet_task_name}_{subset}_{mn}_std": np.std(metric_res[mn]) for mn in metric_names}
                molnet_task_data_avg.update({**average_metrics, **std_metrics})
        molnet_task_data_all.update({"run_name": [run_name]*5})
        df_all = df_all.append(pd.DataFrame(molnet_task_data_all))
        data_avg.append({"run_name": run_name, **molnet_task_data_avg})

    df_avg = pd.DataFrame(data_avg)
    return df_all, df_avg

In [8]:
df_all, df_avg = get_dataframes(cloud_dir)

In [9]:
df_all

,bace_classification_valid_roc_auc_score,bace_classification_valid_average_precision_score,bace_classification_test_roc_auc_score,bace_classification_test_average_precision_score,bace_regression_valid_pearsonr,bace_regression_valid_rmse,bace_regression_test_pearsonr,bace_regression_test_rmse,bbbp_valid_roc_auc_score,bbbp_valid_average_precision_score,...,delaney_test_rmse,lipo_valid_pearsonr,lipo_valid_rmse,lipo_test_pearsonr,lipo_test_rmse,tox21_valid_roc_auc_score,tox21_valid_average_precision_score,tox21_test_roc_auc_score,tox21_test_average_precision_score,run_name
0,0.596748,0.640503,0.757428,0.803327,0.113098,0.494110,0.731038,1.096858,0.970303,0.968082,...,0.461782,0.773668,0.648206,0.719486,0.641808,0.783030,0.453764,0.747493,0.424763,run_0
1,0.596482,0.640416,0.757428,0.803327,0.113863,0.491202,0.729922,1.083382,0.969138,0.968365,...,0.461380,0.773772,0.647530,0.718227,0.642561,0.775867,0.447403,0.754734,0.429691,run_0
2,0.596660,0.641714,0.757428,0.803327,0.113995,0.495492,0.729105,1.098197,0.969332,0.967980,...,0.462781,0.761252,0.660363,0.704606,0.653160,0.781400,0.453767,0.741426,0.429049,run_0
3,0.596482,0.640297,0.757428,0.803327,0.111739,0.494640,0.733085,1.097865,0.969429,0.967529,...,0.458823,0.753817,0.671968,0.695670,0.661683,0.778955,0.448230,0.747444,0.422272,run_0
4,0.596393,0.640210,0.757428,0.803327,0.113524,0.492913,0.731399,1.095938,0.970982,0.970051,...,0.460283,0.780542,0.641785,0.724696,0.639564,0.777968,0.452808,0.744802,0.427032,run_0
0,0.650498,0.692315,0.791486,0.835834,0.172229,0.563354,0.799910,1.104355,0.967682,0.968240,...,0.410833,0.699566,0.731749,0.661364,0.707962,0.763856,0.456702,0.831450,0.420656,run_1
1,0.649076,0.690954,0.791486,0.831017,0.182468,0.565476,0.810622,1.001958,0.968071,0.969398,...,0.414782,0.707672,0.722468,0.665169,0.701355,0.771706,0.465950,0.803219,0.397695,run_1
2,0.651208,0.692552,0.791123,0.835047,0.167925,0.560005,0.803815,1.089828,0.967974,0.969196,...,0.429092,0.702438,0.722310,0.637848,0.722440,0.765743,0.463505,0.821273,0.406365,run_1
3,0.652097,0.690869,0.790399,0.831698,0.164164,0.565016,0.793742,1.118970,0.968071,0.969543,...,0.418384,0.702826,0.727075,0.660596,0.707792,0.780585,0.456097,0.806546,0.396876,run_1
4,0.656716,0.698773,0.796377,0.840267,0.177277,0.564676,0.808512,1.065302,0.968459,0.969447,...,0.423083,0.691847,0.736963,0.636978,0.729956,0.773550,0.463900,0.822692,0.422046,run_1


In [10]:
df_avg

,run_name,bace_classification_valid_roc_auc_score_mean,bace_classification_valid_average_precision_score_mean,bace_classification_valid_roc_auc_score_std,bace_classification_valid_average_precision_score_std,bace_classification_test_roc_auc_score_mean,bace_classification_test_average_precision_score_mean,bace_classification_test_roc_auc_score_std,bace_classification_test_average_precision_score_std,bace_regression_valid_pearsonr_mean,...,lipo_test_pearsonr_std,lipo_test_rmse_std,tox21_valid_roc_auc_score_mean,tox21_valid_average_precision_score_mean,tox21_valid_roc_auc_score_std,tox21_valid_average_precision_score_std,tox21_test_roc_auc_score_mean,tox21_test_average_precision_score_mean,tox21_test_roc_auc_score_std,tox21_test_average_precision_score_std
0,run_0,0.596553,0.640628,0.000131,0.000552,0.757428,0.803327,0.000000,0.000000,0.113244,...,0.010732,0.008398,0.779444,0.451194,0.002526,0.002792,0.747179,0.426561,0.004382,0.002751
1,run_1,0.651919,0.693092,0.002594,0.002922,0.792174,0.834773,0.002139,0.003315,0.172812,...,0.012330,0.010589,0.771088,0.461231,0.005958,0.004036,0.817036,0.408728,0.010570,0.010839
2,run_38,0.529531,0.583894,0.000133,0.000057,0.697029,0.709125,0.000145,0.000088,0.161029,...,0.002542,0.002655,0.747829,0.445990,0.007791,0.010090,0.831469,0.449950,0.002657,0.007874
3,run_39,0.618301,0.691697,0.003983,0.005720,0.782029,0.813169,0.002246,0.000545,0.133880,...,0.008544,0.006314,0.755765,0.464589,0.005916,0.011755,0.804814,0.353405,0.009988,0.010090
4,run_45,0.586958,0.651463,0.000181,0.000281,0.735543,0.810961,0.000072,0.000025,0.205019,...,0.001670,0.002481,0.768102,0.469893,0.002769,0.002208,0.740437,0.363042,0.004185,0.002332


In [11]:
combined_avg_df = pd.merge(left=df, right=df_avg, on='run_name')
# combined_avg_df['run_name'] = combined_avg_df['run_name'].apply(lambda x: f"mlm_{x}")

combined_all_df = pd.merge(left=df, right=df_all, on='run_name')
# combined_all_df['run_name'] = combined_all_df['run_name'].apply(lambda x: f"mlm_{x}")

In [12]:
combined_avg_df

,run_name,min_eval_loss,hidden_size,attention_probs_dropout_prob,hidden_dropout_prob,intermediate_size,num_attention_heads,num_hidden_layers,learning_rate,per_device_train_batch_size,...,lipo_test_pearsonr_std,lipo_test_rmse_std,tox21_valid_roc_auc_score_mean,tox21_valid_average_precision_score_mean,tox21_valid_roc_auc_score_std,tox21_valid_average_precision_score_std,tox21_test_roc_auc_score_mean,tox21_test_average_precision_score_mean,tox21_test_roc_auc_score_std,tox21_test_average_precision_score_std
0,run_39,0.408093,209,0.176,0.128,3968,11,3,0.000002,25,...,0.008544,0.006314,0.755765,0.464589,0.005916,0.011755,0.804814,0.353405,0.009988,0.010090
1,run_1,0.253704,407,0.156,0.295,9044,11,2,0.000005,25,...,0.012330,0.010589,0.771088,0.461231,0.005958,0.004036,0.817036,0.408728,0.010570,0.010839
2,run_45,0.062321,384,0.109,0.144,464,12,3,0.000141,25,...,0.001670,0.002481,0.768102,0.469893,0.002769,0.002208,0.740437,0.363042,0.004185,0.002332
3,run_38,0.316783,126,0.109,0.279,456,3,2,0.000021,25,...,0.002542,0.002655,0.747829,0.445990,0.007791,0.010090,0.831469,0.449950,0.002657,0.007874
4,run_0,0.082666,264,0.224,0.122,4888,8,3,0.000396,25,...,0.010732,0.008398,0.779444,0.451194,0.002526,0.002792,0.747179,0.426561,0.004382,0.002751


In [13]:
combined_all_df

,run_name,min_eval_loss,hidden_size,attention_probs_dropout_prob,hidden_dropout_prob,intermediate_size,num_attention_heads,num_hidden_layers,learning_rate,per_device_train_batch_size,...,delaney_test_pearsonr,delaney_test_rmse,lipo_valid_pearsonr,lipo_valid_rmse,lipo_test_pearsonr,lipo_test_rmse,tox21_valid_roc_auc_score,tox21_valid_average_precision_score,tox21_test_roc_auc_score,tox21_test_average_precision_score
0,run_39,0.408093,209,0.176,0.128,3968,11,3,0.000002,25,...,0.887726,0.473826,0.585601,0.815095,0.538424,0.778248,0.761110,0.474264,0.804981,0.342878
1,run_39,0.408093,209,0.176,0.128,3968,11,3,0.000002,25,...,0.894136,0.464875,0.595695,0.808738,0.558163,0.762622,0.754290,0.447144,0.799795,0.345286
2,run_39,0.408093,209,0.176,0.128,3968,11,3,0.000002,25,...,0.896134,0.462572,0.599250,0.806847,0.558706,0.763863,0.744852,0.454886,0.822888,0.371321
3,run_39,0.408093,209,0.176,0.128,3968,11,3,0.000002,25,...,0.894566,0.460415,0.595595,0.809195,0.557109,0.765294,0.758880,0.468519,0.803660,0.351421
4,run_39,0.408093,209,0.176,0.128,3968,11,3,0.000002,25,...,0.896773,0.461996,0.592402,0.810230,0.543326,0.774747,0.759695,0.478129,0.792749,0.356120
5,run_1,0.253704,407,0.156,0.295,9044,11,2,0.000005,25,...,0.923107,0.410833,0.699566,0.731749,0.661364,0.707962,0.763856,0.456702,0.831450,0.420656
6,run_1,0.253704,407,0.156,0.295,9044,11,2,0.000005,25,...,0.921820,0.414782,0.707672,0.722468,0.665169,0.701355,0.771706,0.465950,0.803219,0.397695
7,run_1,0.253704,407,0.156,0.295,9044,11,2,0.000005,25,...,0.910316,0.429092,0.702438,0.722310,0.637848,0.722440,0.765743,0.463505,0.821273,0.406365
8,run_1,0.253704,407,0.156,0.295,9044,11,2,0.000005,25,...,0.919305,0.418384,0.702826,0.727075,0.660596,0.707792,0.780585,0.456097,0.806546,0.396876
9,run_1,0.253704,407,0.156,0.295,9044,11,2,0.000005,25,...,0.914678,0.423083,0.691847,0.736963,0.636978,0.729956,0.773550,0.463900,0.822692,0.422046


In [15]:
combined_avg_df.to_csv('ft_results_combined.csv', index=False)
combined_all_df.to_csv('ft_results_all_seeds.csv', index=False)